### Imports


In [ ]:
import os
os.chdir('../')

from src.data import constants
import pandas as pd
from pandas import read_csv
import datetime as dt
import numpy as np

from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

### Read data from local source

MIMIC-IV Dataset

Core

In [ ]:
with open(constants.admissions) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_admissions = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)
        
with open(constants.patients) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_patients = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)
        
with open(constants.transfers) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_transfers = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)

core fusion

In [ ]:
df_core_fusion = df_admissions.merge(df_patients, on=("subject_id")).merge(df_transfers, on=('subject_id', 'hadm_id'))

ICU

In [ ]:

with open(constants.d_items) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_d_items = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)
        
with open(constants.procedureevents) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_procedureevents = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)
        
with open(constants.outputevents) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_outputevents = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)
        
with open(constants.inputevents) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_inputevents = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)
        
with open(constants.icustays) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_icustays = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)
        
with open(constants.datetimeevents) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_datetimeevents = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)


In [ ]:
df_chartevents = pd.read_csv(constants.chartevents, dtype={'value': 'object', 'valueuom': 'object'}, nrows=20000000)

In [ ]:
df_chartevents[df_chartevents["subject_id"].isin([10003700])]

In [ ]:
df_chartevents[df_chartevents["subject_id"].isin(constants.cohort)]

icu fusion

In [ ]:
df_icu_fusion = df_d_items.merge(df_procedureevents, on=("subject_id"))\
.merge(df_outputevents, on=('subject_id', 'hadm_id'))\
.merge(df_inputevents, on=('subject_id', 'hadm_id'))\
.merge(df_icustays, on=('subject_id', 'hadm_id'))\
.merge(df_datetimeevents, on=('subject_id', 'hadm_id'))

HOSP

In [ ]:
with open(constants.d_labitems) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_d_labitems = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)
        
with open(constants.d_hcpcs) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_d_hcpcs = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)
        
with open(constants.hcpcsevents) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_hcpcsevents = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)
        


In [ ]:
df_labevents = pd.read_csv(constants.labevents, dtype={'storetime': 'object', 'value': 'object', 'valueuom': 'object', 'flag': 'object', 'priority': 'object', 'comments': 'object'}, nrows=10000000)



In [ ]:
df_labevents[df_labevents["subject_id"].isin(constants.cohort)]

hosp fusion

In [ ]:
df_hosp_fusion = d_labitems.merge(d_hcpcs, on=("subject_id"))\
.merge(hcpcsevents, on=('subject_id', 'hadm_id'))\
.merge(df_labevents, on=('subject_id', 'hadm_id'))

MIMIC-CXR

In [ ]:
with open(constants.mimic_cxr_chexpert) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_mimic_cxr_chexpert = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)
        
with open(constants.mimic_cxr_metadata) as f:
        cols = next(f).strip().split(',')
        S = constants.cohort
        df_mimic_cxr_metadata = pd.DataFrame([l for x in f if (l:=x.rstrip().split(','))[0] in S], columns=cols)

In [ ]:
mimic_cxr_chexpert[mimic_cxr_chexpert["subject_id"].isin(constants.cohort)]

CXR dataset fusion

In [ ]:
df_cxr_fusion = df_mimic_cxr_chexpert.merge(df_mimic_cxr_metadata, on= ('subject_id', 'study_id'))

In [ ]:
df_cxr_fusion.info()

#### Core_Hosp_ICU_CXR Fusion

In [ ]:
df_core_icu_hosp_cxr_fusion = df_core_fusion.merge(df_icu_fusion, on=("subject_id"))\
.merge(df_hosp_fusion, on=("subject_id")).merge(df_cxr_fusion, on=("subject_id"))

In [ ]:
df_core_icu_hosp_cxr_fusion.info()

Demographics categorical features creation

In [ ]:
demo_embeddings = ['anchor_age', 'gender', 'ethnicity', 'marital_status', 'language', 'insurance']

for i in range (len(demo_embeddings)):
      
    df_core_icu_hosp_cxr_fusion['de_'+str(i)] = df_core_icu_hosp_cxr_fusion[demo_embeddings[i]]

Demographics categorical features encoding

In [ ]:
for i in range(5):
          df_core_icu_hosp_cxr_fusion['de_'+str(i+1)] = le.fit_transform(df_core_icu_hosp_cxr_fusion['de_'+str(i+1)])

In [ ]:
df_core_icu_hosp_cxr_fusion["de_5"].unique()

In [ ]:
df_core_icu_hosp_cxr_fusion = df_core_icu_hosp_cxr_fusion.drop(['anchor_age', 'gender', 'ethnicity', 'marital_status', 'language', 'insurance'], axis=1)


In [ ]:
df_core_icu_hosp_cxr_fusion['death_status'] = np.where(df_core_icu_hosp_cxr_fusion['discharge_location'] == 'DIED' ,1,0)


In [ ]:
df_core_icu_hosp_cxr_fusion = df_core_icu_hosp_cxr_fusion.drop(['discharge_location'], axis =1)


In [ ]:
df_core_icu_hosp_cxr_fusion.columns

In [ ]:
df_core_icu_cxr_fusion = df_core_icu_cxr_fusion.loc[: , ['subject_id', 'hadm_id','stay_id',  'admittime','de_0', 'de_1', 'de_2',
       'de_3', 'de_4', 'de_5', 'death_status', 'study_id', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
       'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
       'Pneumonia', 'Pneumothorax', 'Support Devices', 'dicom_id',
       'PerformedProcedureStepDescription', 'ViewPosition', 'Rows', 'Columns',
       'StudyDate', 'StudyTime', 'ProcedureCodeSequence_CodeMeaning',
       'ViewCodeSequence_CodeMeaning',
       'PatientOrientationCodeSequence_CodeMeaning', ]]

In [ ]:
df_core_icu_hosp_cxr_fusion.columns

In [ ]:
len(df_core_icu_hosp_cxr_fusion['subject_id'].unique())

In [ ]:
df_core_icu_hosp_cxr_fusion

# Times series

In [ ]:
# LOAD MASTER DICTIONARY OF MIMIC IV EVENTS

In [ ]:
# Generate dictionary for chartevents, labevents and HCPCS
df_patientevents_categorylabels_dict = pd.DataFrame(columns = ['eventtype', 'category', 'label'])

 
# Load dictionaries
df_d_items = pd.read_csv(constants.d_items)

# Get Chartevent items with labels & category
df = df_d_items
for category_idx, category in enumerate(sorted((df.category.astype(str).unique()))):
    #print(category)
    category_list = df[df['category']==category]
    for item_idx, item in enumerate(sorted(category_list.label.astype(str).unique())):
        df_patientevents_categorylabels_dict = df_patientevents_categorylabels_dict.append({'eventtype': 'chart', 'category': category, 'label': item}, ignore_index=True)

        
        

In [ ]:
def get_chartevent_tsfresh_timeseries_embeddings(dt_patient, df_patientevents_categorylabels_dict, verbose=0):
    # Inputs:
    #   dt_patient -> Timebound Patient ICU stay structure
    #   df_patientevents_categorylabels_dict -> MIMIC IV Event dictionary
    #   verbose -> Flag to print generated outputs (0,1,2)
    #
    # Outputs:
    #   evs_features -> TSfresh generated chart event features for each timeseries
    
    # %% EXAMPLE OF USE
    # evs_features = extract_chartevent_tsfresh_timeseries_embeddings(dt_patient, df_patientevents_categorylabels_dict, event_type, verbose=1)
    
  
    
    # Prep features of empty timeseries features from TSFresh in the context of clinical data
    fc_parameters = {"length": None,
                    "absolute_sum_of_changes": None, 
                    "maximum": None, 
                    "mean": None,
                    "mean_abs_change": None,
                    "mean_change": None,
                    "median": None,
                    "minimum": None,
                    "standard_deviation": None,
                    "variance": None,
                    "large_standard_deviation": [{"r": r * 0.2} for r in range(1, 5)],
                     
                     # Comment by Yu: don't think we need the 1 for quntile?
                    "quantile": [{"q": q} for q in [.25, .5, .75, 1]],
                    "linear_trend": [{"attr": "pvalue"}, {"attr": "rvalue"}, {"attr": "intercept"},{"attr": "slope"}, {"attr": "stderr"}]}
  
    x_hr =[0]
    y_hr = np.arange(len(x_hr))
  
    # Extract Features with TSFresh
    timeseries = pd.DataFrame({'id': np.zeros_like(y_hr), 'valnum': y_hr, 'time': x_hr}, columns=['id', 'valnum', 'time'])
    features_empty_timeseries = extract_features(timeseries, column_id="id", column_sort="time", disable_progressbar=True, default_fc_parameters=fc_parameters)
    
    
    #Get patient events by event type
    evs = dt_patient.chartevents
    
    #List all types of chart events (Charts, Labs and signals)
    for eventtype_idx, eventtype in enumerate(sorted((df_patientevents_categorylabels_dict.eventtype.unique()))):
        if verbose >= 3: print('* ' + eventtype)
        event_list = df_patientevents_categorylabels_dict[df_patientevents_categorylabels_dict['eventtype']==eventtype]
        for category_idx, category in enumerate(sorted((df_patientevents_categorylabels_dict.category.unique()))):
            if verbose >= 3: print('-> ' + category)
            category_list = df_patientevents_categorylabels_dict[df_patientevents_categorylabels_dict['category']==category]
            for item_idx, item in enumerate(sorted(category_list.label.unique())):
                if verbose >= 3: print('---> ' + item) 
                
                # POPULATE FEATURE SPACE FOR PATIENT
                item_chart = evs[evs['label']==item]
                empty_timeseries = False
                # Set x equal to the times
                x_hr = item_chart.deltacharttime[item_chart.label==item]
                if len(x_hr)==0: 
                    x_hr =[0]
                    empty_timeseries = True
                    
                y_hr = item_chart.valuenum[item_chart.label==item]
                y_hr = y_hr[~(np.isnan(y_hr))]
                x_hr = x_hr[0:len(y_hr)]
                if y_hr.empty: 
                    y_hr = np.arange(len(x_hr))
                    extracted_features = features_empty_timeseries
                else:                    
                    # Extract Features with TSFresh
                    timeseries = pd.DataFrame({'id': np.zeros_like(y_hr), 'valnum': y_hr, 'time': x_hr}, columns=['id', 'valnum', 'time'])
                    extracted_features = impute(extract_features(timeseries, column_id="id", column_sort="time", disable_progressbar=disable_progressbar, default_fc_parameters = fc_parameters))
                    
                if (eventtype_idx ==0) & (category_idx ==0) & (item_idx == 0):
                    evs_features = extracted_features
                else:
                    evs_features = evs_features.append(extracted_features) 
                    
    # Transform extracted features from 0-1
    transformer = QuantileTransformer().fit(evs_features)
    norm_evs_features = transformer.transform(evs_features)
    norm_evs_features = np.asarray(norm_evs_features)
    
    if verbose >= 1:
        # Plot feature representation
        plt.figure(figsize = (20,5))
        plt.imshow(X, cmap='hot', interpolation='nearest', aspect='auto')
        plt.colorbar(label="Patient Timeseries Features", orientation="vertical")
        plt.show()
        
    return norm_evs_features, evs_features


def pivot_chartevent(df, event_list):
    # create a new table with additional columns with label list  
    df1 = df[['subject_id', 'hadm_id', 'stay_id', 'charttime']] 
    for event in event_list: 
        df1[event] = np.nan
         #search in the abbreviations column  
        df1.loc[(df['label']==event), event] = df['valuenum'].astype(float)
    df_out = df1.dropna(axis=0, how='all', subset=event_list)
    return df_out 


#FUNCTION TO COMPUTE A LIST OF TIME SERIES FEATURES
def get_ts_emb(df_pivot, event_list):
    # Inputs:
    #   df_pivot -> Pivoted table
    #   event_list -> MIMIC IV Type of Event
    #
    # Outputs:
    #   df_out -> Embeddings
    
    # %% EXAMPLE OF USE
    # df_out = get_ts_emb(df_pivot, event_list)
    
    # Initialize table
    try:
        df_out = df_pivot[['subject_id', 'hadm_id']].iloc[0]
    except:
#         print(df_pivot)
        df_out = pd.DataFrame(columns = ['subject_id', 'hadm_id'])
#         df_out = df_pivot[['subject_id', 'hadm_id']]
        
     #Adding a row of zeros to df_pivot in case there is no value
    df_pivot = df_pivot.append(pd.Series(0, index=df_pivot.columns), ignore_index=True)
    
    #Compute the following features
    for event in event_list:
        series = df_pivot[event].dropna() #dropna rows
        if len(series) >0: #if there is any event
            df_out[event+'_max'] = series.max()
            df_out[event+'_min'] = series.min()
            df_out[event+'_mean'] = series.mean(skipna=True)
            df_out[event+'_variance'] = series.var(skipna=True)
            df_out[event+'_meandiff'] = series.diff().mean() #average change
            df_out[event+'_meanabsdiff'] =series.diff().abs().mean()
            df_out[event+'_maxdiff'] = series.diff().abs().max()
            df_out[event+'_sumabsdiff'] =series.diff().abs().sum()
            df_out[event+'_diff'] = series.iloc[-1]-series.iloc[0]
            #Compute the n_peaks
            peaks,_ = find_peaks(series) #, threshold=series.median()
            df_out[event+'_npeaks'] = len(peaks)
            #Compute the trend (linear slope)
            if len(series)>1:
                df_out[event+'_trend']= np.polyfit(np.arange(len(series)), series, 1)[0] #fit deg-1 poly
            else:
                 df_out[event+'_trend'] = 0
    return df_out


def get_ts_embeddings(dt_patient, event_type):
    # Inputs:
    #   dt_patient -> Timebound Patient ICU stay structure
    #
    # Outputs:
    #   ts_emb -> TSfresh-like generated Lab event features for each timeseries
    #
    # %% EXAMPLE OF USE
    # ts_emb = get_labevent_ts_embeddings(dt_patient)
    
    #Get chartevents
        
    elif(event_type == 'chart'):
        df = dt_patient.chartevents
        #Define chart events of interest
        event_list = ['Heart Rate','Non Invasive Blood Pressure systolic',
                    'Non Invasive Blood Pressure diastolic', 'Non Invasive Blood Pressure mean', 
                    'Respiratory Rate','O2 saturation pulseoxymetry', 
                    'GCS - Verbal Response', 'GCS - Eye Opening', 'GCS - Motor Response'] 
        df_pivot = pivot_chartevent(df, event_list)
    
    #Pivote df to record these values
    
    ts_emb = get_ts_emb(df_pivot, event_list)
    try:
        ts_emb = ts_emb.drop(['subject_id', 'hadm_id']).fillna(value=0)
    except:
        ts_emb = pd.Series(0, index=ts_emb.columns).drop(['subject_id', 'hadm_id']).fillna(value=0)

    return ts_emb

In [ ]:
def process_cxr_embeddings_haim_id(haim_id, dt_patient, df_init):
    # Time Series (TSFRESH-like) CHARTEVENT & LABEVENT EMBEDDINGS EXTRACTION
    aggregated_ts_ce_embeddings = get_ts_embeddings(dt_patient, event_type = 'chart')
    gc.collect() #Clear memory
    
    aggregated_ts_le_embeddings = get_ts_embeddings(dt_patient, event_type = 'lab')
    gc.collect() #Clear memory
    
    aggregated_ts_pe_embeddings = get_ts_embeddings(dt_patient, event_type = 'procedure')
    gc.collect() #Clear memory
    
        # Create Dataframes filteed by ordered sample number for Fusion
    df_haim_ids_fusion = pd.DataFrame([haim_id],columns=['haim_id'])
    df_ts_ce_embeddings_fusion = pd.DataFrame(aggregated_ts_ce_embeddings.values.reshape(1,-1), columns=['ts_ce_'+str(i) for i in range(aggregated_ts_ce_embeddings.values.shape[0])])
    df_ts_le_embeddings_fusion = pd.DataFrame(aggregated_ts_le_embeddings.values.reshape(1,-1), columns=['ts_le_'+str(i) for i in range(aggregated_ts_le_embeddings.values.shape[0])])
    df_ts_pe_embeddings_fusion = pd.DataFrame(aggregated_ts_pe_embeddings.values.reshape(1,-1), columns=['ts_pe_'+str(i) for i in range(aggregated_ts_pe_embeddings.values.shape[0])])
    
    
    # Embeddings FUSION
    df_fusion = df_haim_ids_fusion
    df_fusion = pd.concat([df_fusion, df_init], axis=1)

    df_fusion = pd.concat([df_fusion, df_ts_ce_embeddings_fusion], axis=1)
    df_fusion = pd.concat([df_fusion, df_ts_le_embeddings_fusion], axis=1)
    df_fusion = pd.concat([df_fusion, df_ts_pe_embeddings_fusion], axis=1)
    
   
    #Add targets
    df_fusion = pd.concat([df_fusion, df_vision_targets_fusion], axis=1)
    gc.collect() #Clear memory
    
    return df_fusion

In [ ]:
"""# Let's select a single HAIM Patient from pickle files and check if it fits inclusion criteria
haim_patient_idx = 4

#Load precomputed file
filename = f"{haim_patient_idx:08d}" + '.pkl'
folder = f"{haim_patient_idx:05d}"[:2] + "/"
patient = load_patient_object(core_mimiciv_path + 'pickle/folder' + folder + filename)

# Get information of chest x-rays conducted within this patiewnt stay
df_cxr = patient.cxr
df_imcxr = patient.imcxr
admittime = patient.admissions.admittime.values[0]
dischtime = patient.admissions.dischtime.values[0]
df_stay_cxr = df_cxr.loc[(df_cxr['charttime'] >= admittime) & (df_cxr['charttime'] <= dischtime)]

if not df_stay_cxr.empty:
    for idx, df_stay_cxr_row in df_stay_cxr.iterrows():
        # Get stay anchor times
        img_charttime = df_stay_cxr_row['charttime']
        img_deltacharttime = df_stay_cxr_row['deltacharttime']

        # Get time to discharge and discharge location/status
        img_id = df_stay_cxr_row["dicom_id"]
        img_length_of_stay = date_diff_hrs(dischtime, img_charttime)
        discharge_location = patient.core['discharge_location'][0]
        if discharge_location == "DIED": death_status = 1
        else: death_status = 0
            
        # Select allowed timestamp range
        start_hr = None
        end_hr = img_deltacharttime
        
        # We need to reload it since the original object has been modified
        patient = load_patient_object(core_mimiciv_path + 'pickle/folder' + folder + filename)
        dt_patient = get_timebound_patient_icustay(patient, start_hr , end_hr)
        is_included = True

        if is_included:
            df_init = pd.DataFrame([[img_id, img_charttime, img_deltacharttime, discharge_location, img_length_of_stay, death_status]],columns=['img_id', 'img_charttime', 'img_deltacharttime', 'discharge_location', 'img_length_of_stay', 'death_status'])
            df_fusion = process_cxr_embeddings_haim_id(haim_id, dt_patient, df_init)
            
            if os.path.isfile(fname):
                df_fusion.to_csv(fname, mode='a', index=False, header=False)
            else:
                df_fusion.to_csv(fname, mode='w', index=False)"""

In [ ]:
#Sample of 10 patients

#sample = read_csv(constants.icu_cxr_patients_sample10)

#df_core_icu_cxr_fusion_sample = df_core_icu_cxr_fusion.loc[df_core_icu_cxr_fusion['subject_id'].isin(sample)]

'''
0    14811141
1    18874374
2    11272213
3    13762583
4    18087960
5    13500443
6    12189736
7    14024750
8    19136566
9    18481208'''

In [ ]:
df_core_icu_cxr_fusion_sample